<a id='top'></a>

#  IBM Q Apply Jupyter Notebook
Author: Christian Lenke, IBM<br>
Version: 2020-11-09

## Table of content

* [Preface](#intro)
* [Introduction](#intro)
<p>
* [Installation, import, function definition](#preparation)
  * [Environment Definition and Customization](#customization)
  * [Get a connection to the Q Apply Server](#connect_qapp)
<p>
* [Q Apply Status overview](#overview_qapp)
  * [General Q Apply overview](#overview_qapp)
  * [Receive Queue state](#recvq)
  * [Q Subscription state by Receive Queue](#qsubs_recvq)    
  * [Q Subscription type by Receive Queue](#qsubs_type)        
<p>
* [Q Apply performance graphs](#qapp_performance_hour)
  * [APPLYMON KPIs per hour](#qapp_performance_hour)    
  * [APPLYMON KPIs per day](#qapp_performance_day)        
  * [Detailed APPLYMON KPIs (one hour of data)](#qapp_performance_details)            
<p>
* [Status details](#qapp_details)
  * [Receive Queue details](#qapp_details)
  * [Q Subscription details](#qsubs_details)
  * [Q Subscription quality assurance](#qsubs_qa)        
<p>
* [Q Apply Control Tables](#qapp_cntl)
  * [APPLYPARMS](#qapp_cntl)
  * [APPLYMON](#qapp_mon)
  * [APPLYTRACE](#qapp_trace)            

[Back to Top](#top)
<a id='intro'></a>

## Preface

This Jupyter Notebook is sample code. No warranty.

## Introduction

This set of Jupyter Notebook can be used to display status, performance, and health of an IBM Q Replication setup. The following components are provided:
 <ul>
  <li>Q Capture Jupyter Notebook</li>
  <li>Q Apply Jupyter Notebook (this Notebook)</li>
  <li>Q Replication function library Jupyter Notebook</li>
  <li>A set of SQL queries in files (asnmonitor) for status calculation and quality assurance</li>    
</ul> 
The Notebooks display the content of the IBM Q Replication control tables. This includes an overall health summary for both Q Capture and Q Apply, performance graphs (throughput, latency, etc.), and control table details for both queues and subscriptions.<br>
Start with the following:<br>
 <ol>
  <li>Copy the set of SQL query files to a location in your environment</li>
  <li>Cuctomize some of the SQL files depending on your environment (more details in the Q Replication function library Notebook)</li>
  <li>Customize the Q Replication function library (details explained in that Notebook)</li>
  <li>Customize the Q Capture and Q Apply Notebook (this Notebook) in section "Environment Definition and Customization" 
</ol> 
To better understand the Q Replication control tables and the SQL queries used, have a look at<br>
<A HREF="https://developer.ibm.com/recipes/tutorials/q-replication-for-dbas/">Q Replication for DBAs</A>,<br>
<A HREF="https://www.ibm.com/support/knowledgecenter/en/SSTRGZ_11.4.0/com.ibm.swg.im.iis.repl.qrepl.doc/topics/iiyrqctbrcaplist.html">Control tables at the Q Capture server</A>,<br> 
<A HREF="https://www.ibm.com/support/knowledgecenter/en/SSTRGZ_11.4.0/com.ibm.swg.im.iis.repl.qrepl.doc/topics/iiyrqctbrapplist.html">Control tables at the Q Apply server</A>, or <br>
<A HREF="https://www.ibm.com/support/knowledgecenter/en/SSTRGZ_11.4.0/com.ibm.swg.im.iis.db.repl.intro.doc/topics/iiyrqinfroadmap.html">Q Replication Information Roadmap (IBM Knowledge Center)</A>

[Back to Top](#top)
<a id='preparation'></a>

# Preparation

## Installation, import, function definition 

First of all the "IBM Q Replication Monitoring Jupyter Library.ipynb" is run. It imports all required libraries (SQLAlchemy, pandas, pixiedust, ...) and defines functions used in this Notebook.

In [ ]:
%run "IBM Q Replication Monitoring Jupyter Library.ipynb"

[Back to Top](#top)
<a id='customization'></a>

## Environment Definition and Customization

BEFORE YOU START define your environment here. Specify your individual values for all variables in the below cell.

In [ ]:
# Specifying the Db2 driver (don't change)
db2_driver='ibm_db_sa'

# Define your Q Apply Schema. All queries selecting data from the Q Replication control tables 
# have been coded schemaless, the schema needs to be declared once here.
apply_schema = 'ASN'
# apply_schema = 'LSN'

# Database alias of the Q Apply Server (Db2 LUW database name or Db2 z/OS location name)
# db2alias = 'BLU_TGT'
# db2alias = 'ASCIIDB'
db2alias = 'TARGETDB'

# Setting the connection variables for the Q Apply server (usually the Q Replication target database)
# Host name or ip-address of the Q Apply Server
db2host  = 'localhost'
# Db2 port of the Q Apply Server (defined as char)
db2port  = 50000

# User id privileged to read the Q Apply control tables
db2user  = 'REPLADM'
# You will be prompted to type in your password
db2password = getpass.getpass('Password for database ' + db2alias + ': '); 

In [ ]:
# Switch to control debug messages. False: No debug messages; True: Some debug messages will be printed
# debug = True
debug = False

# Display all columns when showing a data frame
pandas.set_option('display.max_columns', None)

# Define the date range for plotting performance data from IBMQREP_APPLYMON (plots grouped by hour)
# Default: 60 (Retrieve only 60 days of Monitor data for plotting)
# If you want to retrieve all available data, set max_mon_plot_range = -1
# max_mon_plot_range = -1
max_mon_plot_range = 60

# Define the start date for plotting the IBMQREP_APPLYMON details (ungrouped)
# One hour of APPLYMON data will be plotted, starting from the defined start_ts here
# Default: 'max' (Last hour of available data.) 
# A dedicated Db2 timestamp can be specified alternatively
# start_ts_mon_plot_details = 'max'
start_ts_mon_plot_details = '2020-10-28-17.10.00.000000'

# Define the number of most recent messages messages to be retrieved from IBMQREP_APPLYTRACE. 
# Default: 200
# If you want to retrieve all available data, set num_messages_trace = -1
num_messages_trace = -1
#num_messages_trace = 200

# Define the date range for fetching monitor information from IBMQREP_APPLYMON (display as table),
# specified as number of days. Default: 10  
default_mon_date_range = 10
# Custom range: -1 means from oldest available (start) to most recent (end).
# Alternatively, specify timestamp in Db2 format
# custom_mon_start_date  = '2018-11-02-10.00.00.000000'
custom_mon_start_date  = -1
custom_mon_end_date    = -1

In [ ]:
# Check the validity of some parms
check_minus_1_or_positive_int('max_mon_plot_range', max_mon_plot_range)        
check_minus_1_or_positive_int('num_messages_trace', num_messages_trace)        
check_positive_int('default_mon_date_range', default_mon_date_range) 

[Back to Top](#top)
<a id='connect_qapp'></a>

## Get a connection to the Q Apply Server

Connect to the Q Apply Server.

In [ ]:
connection = get_connection(db2_driver, db2user, db2password, db2host, str(db2port), db2alias)

Set the CURRENT SCHEMA to the APPLY_SCHEMA. All queries executed thereafter are schemaless

In [ ]:
# For the connection the SCHEMA is set to the Q Apply schema
connection.execute("SET CURRENT SCHEMA = '" + apply_schema + "'")

print("INFO: CURRENT SCHEMA = '" + apply_schema + "'")

[Back to Top](#top)
<a id='overview_qapp'></a>

# Q Apply Status overview

## General status overview

The first query displays the overall status of the Q Apply Server. The result set displays messages of the following types:
 <ul>
  <li>A-LAT: Is Q Apply running or not / is the latency as expected (threshold needs to be set in 'qrep_monitor_apply.sql')</li>
    <ul>    
        <li>ERROR: Q Apply not running or latency error threshold exceeded</li>
        <li>WARNING: Q Apply latency warning threshold exceeded</li>
        <li>INFO: Q Apply up and running and latency threshold ok</li>        
    </ul>            
  <li>A-RQU: Receive queue status, displayed for each receive queue.</li>
    <ul>    
        <li>ERROR: Receive Queue inactive due to an error</li>
        <li>INFO: Receive Queue active</li>            
    </ul>                    
  <li>A-SUB: Subscription status.</li>
    <ul>    
        <li>ERROR: Subscription not active</li>
    </ul>                    
</ul> 

In [ ]:
# The function get_apply_status executes the query 'qrep_monitor_apply.sql' 
# (with language dependent result set) and returns a pandas data frame
df_appstate = get_apply_status(connection)

# drop column 'CURRENT_SERVER'
df_appstate.drop(df_appstate.columns[2], axis=1, inplace=True)

# conditional formatting for column 'sev'
df_appstate = (df_appstate.style
    .applymap(sev_background, subset=['sev'])
    .applymap(sev_foreground, subset=['sev'])
)

# printing the data frame
df_appstate

[Back to Top](#top)
<a id='recvq'></a>

### Overall receive queue state

Number of active receive queues (green) vs. number of inactive receive queues (red)

In [ ]:
# The function get_apply_status executes a query which counts active vs. inactive queues
# and returns a pandas data frame
df_qstate = get_recvq_state(connection)  

df_qstate.set_index('queues',inplace=True)

# plotting a bar chart
qpl = df_qstate.plot.barh(stacked=True,color=['green', 'red'], 
                          title="Active vs. inactive receive queues",
                          figsize=(10,2))

qpl.set_xlabel("Number of queues")
myvar = qpl.set_ylabel("")

[Back to Top](#top)
<a id='qsubs_recvq'></a>

## Subscription STATE by receive queue

Number of active subscriptions (green) vs. number of inactive subscriptions (red) per receive queue

In [ ]:
# The function get_substate_by_recvq executes a query which counts active vs. inactive queues
# and returns a pandas data frame
df_q = get_substate_by_recvq(connection)

# Determination of the number of distinct receive queues in the result set
numqs = len(df_q.recvq.unique())

# The hight of the following bar charts depends on the number of distinct receive queues.
# The more queues, the less space per individual queue (to limit the size of the plot)
if numqs >=20:
    calc_fig_height = numqs / 2
if numqs <=2:
    calc_fig_height = numqs * 2
else:
    calc_fig_height =  numqs

df_q.set_index('recvq',inplace=True) 

# plotting a bar chart
sqpl = df_q.plot.barh(stacked=True,color=['green', 'red'],
                       title="Number of active vs. inactive subs per receive queue",
                       figsize=(15,calc_fig_height))

sqpl.set_xlabel("Number of subs")
myvar = sqpl.set_ylabel("")

[Back to Top](#top)
<a id='qsubs_type'></a>

## Subscription TYPE Overview

Number of Q Subscription target types per Receive Queue

In [ ]:
# The function get_subtype_by_recvq executes a query which counts available subscriptions 
# by type and returns a pandas data frame
df_st = get_subtype_by_recvq(connection)

# DEBUG - print sample data of the data frame retrieved from APPLYMON
if debug:
    print(df_st)

In [ ]:
# Determination of the number of distinct receive queues in the result set
numqs = len(df_st.recvq.unique())

# The following plot prints a pie for each queue. Two queues in the same row (always fig with 2 columns)
# With anaconda 3 'round' does an closest even round. round(1.5)=2 and round(2.5)=2. Therefore useless here.
# figrows = round(numqs / 2)
# The number of rows is the rounded number of queues devided by 2.
figrows = int((numqs/2)+0.5)

print('INFO: Figure will have ' + str(numqs) + ' pie chart(s) in ' + str(figrows) + ' row(s) and ' 
                                + '2 columns')

In [ ]:
# The function plot_types creates a figure with n donut plots (n = number of queues) to display 
# the different subscription types per queue
subtypeplot = plot_types(df_st,numqs,figrows)
subtypeplot.show()

[Back to Top](#top)
<a id='qapp_performance_hour'></a>

# Q Apply Performance Graphs

## Apply throughput statistics per hour (rows_processed, latency, monster trans)

The following codes creates plots which display Q Replication performance KPIs grouped by hour. First, all data available in IBMQREP_APPLYMON is evaluated (can be limited with parameter <i>max_mon_plot_range</i>). Next, the most recent 20 days will be evaluated, lastly the most recent 5 days. For each section (all, 20 days, 5 days) a separate plot is shown for each receive queue.<p>
The plots show the maximum latency values seen in the particular hour, and the summary of all processed rows.

In [ ]:
# The function get_perf_applymon executes a query which selects data from IBMQREP_APPLYMON
# and returns a pandas data frame. The APPLYMON data is GROUPed BY hour(monitor_time)
# by the query. In case max_mon_plot_range == -1, the query will retrieve all available data 
# from APPLYMON. If max_mon_plot_range > 0, the APPLYMON data will be limited to the recent
# max_mon_plot_range days

if max_mon_plot_range == -1:              
    print('INFO: Evaluating all available APPLYMON data') 
else:   
    print('INFO: Limiting the APPLYMON data to ' + str(max_mon_plot_range) + ' days.')
              
applymon_lat = get_perf_applymon(connection, max_mon_plot_range)

In [ ]:
# DEBUG - print sample data of the data frame retrieved from APPLYMON
if debug:
    print(applymon_lat)

In [ ]:
# Determine the date of max(monitor_time). This is used when only a subset (e.g., most recent
# 5 days of data) is displayed later. Could be that the Notebook is used to evaluate saved 
# APPLYMON data, so that today() - n days would have no data. 
maxmontime = applymon_lat['monitor_date'].max()
maxmondate = datetime.strptime(maxmontime[:10], "%Y-%m-%d")

minmontime = applymon_lat['monitor_date'].min()
minmondate = datetime.strptime(minmontime[:10], "%Y-%m-%d")

if debug:
    print('DEBUG: minmondate.global=' + str(minmondate)[:10])
    print('DEBUG: maxmondate.global=' + str(maxmondate)[:10])
    print()

# Determination of the number of distinct receive queues in the result set
numqs = len(applymon_lat.recvq.unique())

print('INFO: Number of queues=' + str(numqs))

# DEBUG - print the distinct names of the queues
if debug:
   for i in range(0,numqs):
        minmontimeq = applymon_lat[applymon_lat.recvq == applymon_lat.recvq.unique()[i]]['monitor_date'].min()
        maxmontimeq = applymon_lat[applymon_lat.recvq == applymon_lat.recvq.unique()[i]]['monitor_date'].max()
        minmondateq = datetime.strptime(minmontimeq[:10], "%Y-%m-%d")                
        maxmondateq = datetime.strptime(maxmontimeq[:10], "%Y-%m-%d")        
        print('INFO: ' + applymon_lat.recvq.unique()[i] + ' [' + str(minmondateq)[:10] + ' - ' + str(maxmondateq)[:10] + ']') 

All available data from IBMQREP_APPLYMON (max. <i>max_mon_plot_range</i> days)

In [ ]:
# Plot figure (one plot per queue) with all available data   

# deltadays = -1 means all available data of the data frame
deltadays = max_mon_plot_range
if deltadays == -1:
    date_max_days_ago = minmondate
else:
    date_max_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
    if deltadays == -1:
        print('INFO: Plotting range: All available data from ' + minmondate.strftime("%Y-%m-%d") 
                                                           + ' up to ' + maxmondate.strftime("%Y-%m-%d"))
    else:
        print('DEBUG: date_max_days_ago.strftime("%Y-%m-%d"): ' + date_max_days_ago.strftime("%Y-%m-%d"))        
        print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                           + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                           + maxmondate.strftime("%Y-%m-%d"))

# The function plot_lat creates a figure with n plots (n = number of queues) from the 
# data frame applymon_lat
# The hight of the following figures (calculated in plot_lat) depends on the number of distinct 
# receive queues. The more queues, the less space per individual queue (to limit the size of the figure)
if numqs == 0:
    # Data frame could be empty
    print('WARNING: No data available to display.')
else:    
    applymon_plt60 = plot_lat(applymon_lat,numqs,deltadays,'by_hour')
    applymon_plt60.show()       

Recent 20 days from IBMQREP_APPLYMON

In [ ]:
# Zoom in - Same figure as above, with data of the last 20 days only (maxmondate - 20 days)

# display 20 days of data only
deltadays = 20
# old: date_20_days_ago = date.today() - timedelta(days=deltadays)
date_20_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
   print('DEBUG: date_20_days_ago.strftime("%Y-%m-%d"): ' + date_20_days_ago.strftime("%Y-%m-%d"))
   print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                           + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                           + maxmondate.strftime("%Y-%m-%d"))
    
# New data frame - limited by the calculated data range
#   - reset_index was introduced to guarantee xticks in the plot
applymon_lat_20 = applymon_lat[applymon_lat.monitor_date 
                               >= date_20_days_ago.strftime("%Y-%m-%d")].reset_index(drop=True)

# The limited data could have a different number of queues
# calc_fig_height remains as calculated before
numqs = len(applymon_lat_20.recvq.unique())

if numqs == 0:
    # Data frame could be empty
    print('WARNING: No data available to display with MONITOR_TIME > ' + date_20_days_ago.strftime("%Y-%m-%d"))
else:
    applymon_plt20 = plot_lat(applymon_lat_20,numqs,deltadays,'by_hour')
    applymon_plt20.show()        

Recent 5 days from IBMQREP_APPLYMON

In [ ]:
# Zoom in - Same figure as above, with data of the last 5 days only (maxmondate - 5 days)

# display 5 days of data only
deltadays = 5

# old: date_5_days_ago = date.today() - timedelta(days=deltadays)
date_5_days_ago = maxmondate - timedelta(days=deltadays)

# Debug
if debug:
    print('DEBUG: date_5_days_ago.strftime("%Y-%m-%d"): ' + date_5_days_ago.strftime("%Y-%m-%d"))
    print('INFO: Plotting range: Maximum of most recent '  + str(deltadays) + ' days between ' 
                                                           + minmondate.strftime("%Y-%m-%d") + ' and ' 
                                                           + maxmondate.strftime("%Y-%m-%d"))


# New data frame - limited by the calculated data range 
#   - reset_index was introduced to guarantee xticks in the plot
applymon_lat_5 = applymon_lat[applymon_lat.monitor_date 
                              >= date_5_days_ago.strftime("%Y-%m-%d")].reset_index(drop=True)  

# The limited data could have a different number of queues
# calc_fig_height remains as calculated before
numqs = len(applymon_lat_5.recvq.unique())

if numqs == 0:
    # Data frame could be empty
    print('WARNING: No data available to display with MONITOR_TIME > ' + date_5_days_ago.strftime("%Y-%m-%d"))
else:    
    applymon_plt5 = plot_lat(applymon_lat_5,numqs,deltadays,'by_hour')
    applymon_plt5.show()    

[Back to Top](#top)
<a id='qapp_performance_day'></a>

## Apply throughput statistics per day - all available data from APPLYMON (rows_processed, latency)

The following codes creates plots which display Q Replication performance KPIs grouped by day. All data available in IBMQREP_APPLYMON is evaluated. The plots show the average latency values for a day, and the summary of all processed rows. A separate plot is shown for each receive queue,

In [ ]:
# The function get_perf_applymon_day executes a query which selects data from IBMQREP_APPLYMON
# and returns a pandas data frame. The APPLYMON data is GROUPed BY day(monitor_time)
# by the query. The query will retrieve all available data from APPLYMON.

print('INFO: Evaluating all available APPLYMON data') 
              
applymon_lat_day = get_perf_applymon_day(connection)

In [ ]:
# DEBUG - print sample data of the data frame retrieved from APPLYMON
if debug:
   print(applymon_lat_day)

In [ ]:
# Determination of the number of distinct receive queues in the result set
numqs = len(applymon_lat_day.recvq.unique())

print('INFO: Number of queues=' + str(numqs))

# DEBUG - print the number of queues and the distinct names of the queues
if debug:
   for i in range(0,numqs):
       print('DEBUG: ' + applymon_lat_day.recvq.unique()[i])

In [ ]:
# Plot figure (one plot per queue) with all available data   

deltadays = -1

# The function plot_lat creates a figure with n plots (n = number of queues) from the 
# data frame applymon_lat
# The hight of the following figures (calculated in plot_lat) depends on the number of distinct 
# receive queues. The more queues, the less space per individual queue (to limit the size of the figure)
if numqs == 0:
    # Data frame could be empty
    print('No data available to display')
else:    
    applymon_plt_day = plot_lat(applymon_lat_day,numqs,deltadays,'by_day')
    applymon_plt_day.show() 

[Back to Top](#top)
<a id='qapp_performance_details'></a>

## Detailed Apply throughput statistics (APPLYMON) - one hour of available data

In [ ]:
# The function get_perf_applymon_detail executes a query which selects data from IBMQREP_APPLYMON
# and returns a pandas data frame. The query will retrieve one hour of data from APPLYMON, starting 
# at start_ts_mon_plot_details (defined in the defaults section of this Notebook). If 
# start_ts_mon_plot_details == 'max', the last hour of available data will be analyzed

applymon_details = get_perf_applymon_detail(connection, start_ts_mon_plot_details)
mi = get_monitor_interval(connection)

In [ ]:
# DEBUG - print sample data of the data frame retrieved from APPLYMON
if debug:
   print(applymon_details)

In [ ]:
# Determination of the number of distinct receive queues in the result set
numqs = len(applymon_details.recvq.unique())

print('INFO: Number of queues=' + str(numqs))

# DEBUG - print the number of queues and the distinct names of the queues
if debug:
    for i in range(0,numqs):
       print('DEBUG: ' + applymon_details.recvq.unique()[i])

In [ ]:
# Plot figure (one plot per queue), 1 hour of data
print('INFO: MONITOR_INTERVAL=' + str(mi) + ' milliseconds (' + str(mi/1000) + ' seconds)')

# The function plot_lat_details creates a figure with n plots (n = number of queues) from the 
# data frame applymon_details
# The hight of the following figures (calculated in plot_lat_details) depends on the number of distinct 
# receive queues. The more queues, the less space per individual queue (to limit the size of the figure)
if numqs == 0:
    # Data frame could be empty
    print('No data available to display with MONITOR_TIME > ' + start_ts_mon_plot_details)
else:    
    applymon_plt_details = plot_lat_details(applymon_details,numqs,start_ts_mon_plot_details,'by_interval')
    applymon_plt_details.show()

[Back to Top](#top)
<a id='qapp_details'></a>

# Q Apply details

## Receive Queue details

Receive queue details (IBMQREP_RECVQUEUES)

In [ ]:
# The function get_recvq_details executes a query which selects detail data from IBMQREP_RECVQUEUES
# and returns a pandas data frame
df_queues = get_recvq_details(connection, apply_schema)

df_queues.set_index(['sendq'],inplace=True)
df_queues.sort_values(['sendq'], ascending=[True], inplace=True)

# printing the data frame
df_queues

[Back to Top](#top)
<a id='qsubs_details'></a>

## Subscription details

Subscription details (IBMQREP_TARGETS). Use pixiedust options to filter the result set or to convert the table into a  graph.

In [ ]:
# The function get_targets_details executes a query which selects detail data from IBMQREP_TARGETS
# and returns a pandas data frame
df_subs = get_targets_details(connection, apply_schema)

df_subs.sort_values(['subname'], ascending=[True], inplace=True)    

# displaying the data frame using pixiedust
display(df_subs)

[Back to Top](#top)
<a id='qsubs_qa'></a>

## Subscription quality assurance

Runtime error prevention for existing subscriptions. The following potential error situations are reported:
 <ul>
  <li>A-TNF - Target table not found</li>
  <li>A-CNF - Subscribed column does not exist in DB2</li>
  <li>A-CNS - Existing target column not subscribed</li>
  <li>A-GRA - Target table grant missing for Apply user</li>
  <li>A-RIP - Sub for RI parent of a replicated RI child missing</li>
  <li>A-RIC - Sub for RI child of a replicated RI parent missing</li>
  <li>A-BID - Before image column has different data type than after image column</li>
</ul> 

In [ ]:
# The function get_apply_anomylies executes the query 'qrep_check_subs_apply.sql' 
# (with language dependent result set) and returns a pandas data frame
df_appqa = get_apply_anomylies(connection)

df_appqa.drop(df_appqa.columns[2], axis=1, inplace=True)

# conditional formatting for column 'sev'
df_appqa = (df_appqa.style
    .applymap(sev_background, subset=['sev'])
    .applymap(sev_foreground, subset=['sev'])
)

# printing the data frame
df_appqa

[Back to Top](#top)
<a id='qapp_cntl'></a>

# Q Apply Control Tables

## Q Apply parameters

Stored Q Apply parameters (IBMQREP_APPLYPARMS)

In [ ]:
# The function get_applyparms executes a query which selects detail data from IBMQREP_APPLYPARMS
# and returns a pandas data frame
df_applyparms = get_applyparms(connection, apply_schema)

# printing the data frame
df_applyparms

[Back to Top](#top)
<a id='qapp_mon'></a>

## Q Apply APPLYMON

In [ ]:
# Calculate the monitoring date range
# - as lowest date:
#      - in case custom_mon_start_date == -1 take current date - daterange days, otherweise custom_mon_start_date
# - as highest date:
#      - in case custom_mon_end_date == -1 take current date, otherweise custom_mon_start_date

# The function calc_mon_start_end calculates the interval to retrieve from APPLYMON from 
# custom_mon_start_date, custom_mon_end_date, default_mon_date_range. It returns an array with 2 timestamps
calc_mon_dates = calc_mon_start_end(custom_mon_start_date, custom_mon_end_date, default_mon_date_range)

mon_start_date = calc_mon_dates[0]
mon_end_date = calc_mon_dates[1]

# Debug
if debug:
    print("Monitor Start Date: " + mon_start_date)
    print("Monitor End Date:   " + mon_end_date)

In [ ]:
# The function get_applymon executes a query which selects detail data from IBMQREP_APPLYMON
# and returns a pandas data frame
df_applymon = get_applymon(connection, apply_schema, mon_start_date, mon_end_date)

df_applymon.set_index('monitor_time')

# sort does not seem to work
df_applymon.sort_values(['monitor_time'], ascending=False, inplace=True)

# displaying the data frame using Pixiedust
display(df_applymon)

[Back to Top](#top)
<a id='qapp_trace'></a>

## Q Apply message log

Displaying the n (num_messages_trace) most recent runtime log messages from IBMQREP_APPLYTRACE 

In [ ]:
# The function get_applytrace executes a query which selects detail data from IBMQREP_APPLYTRACE
# and returns a pandas data frame
df_applytrace = get_applytrace(connection, apply_schema, num_messages_trace)

# sort does not seem to work
df_applytrace.sort_values(['trace_time'], ascending=False, inplace=True)

# displaying the data frame using Pixiedust
display(df_applytrace)

[Back to Top](#top)
<a id='end'></a>

Done.

## Release Notes

<b>08.05.2019:</b> This is the initial release of the Jupyter Notebook for Q Apply monitoring. It is the beginning of a learning curve and uses various Python and Jupyter Notebook techniques such as:
 <ul>    
   <li>Db2 Python libraries</li>
   <li>Pandas data frames</li>
   <li>Pixiedust for result set visualization</li>
   <li>matplotlib for the graphical representation of performance statistics</li>
 </ul>    
<b>17.06.2019:</b> Added another plot: one hour of detailed APPLYMON data. By default, the most recent hour of available APPLYMON data is displayed. If you want to display a certain hour, change the variable <i>start_ts_mon_plot_details</i> in the customization section at the top of the Notebook.<p>
<b>12.07.2019:</b> Fixed not displayed xticks for plots with limited date ranges (20 days, 5 days) by resetting the index after filtering the data frame (<i>.reset_index(drop=True)</i>)<p>
<b>23.07.2019:</b> Added another plot: Donut charts per receive queue to visualize the different subsciption types
per queue.<p>
<b>02.12.2019:</b> Some queries (e.g., 'get_recvq_details') just retrieve the content of IBMQREP control tables. Previously, these SQL queries contained a list of columns (all columns). This was changed for multiple queries to first generate the list of columns from the Db2 catalog and then build and execute the query. This increases the robustness of the Notebook because with that feature it supports multiple control table architecture levels (prevents errors because of missing (optional) columns).<p>
<b>12.12.2019:</b> Performance KPIs by day - changed from max latency to average latency<p>
<b>09.11.2020:</b> Performance KPIs by day - changed back to max latency<br>
<b>09.11.2020:</b> Improved DEBUG messages

## Feedback

Feedback and ideas for improvement are welcome and can be sent to clenke@de.ibm.com